In [1]:
import pandas as pd
import numpy as np

## Read data

In [2]:
general    = pd.read_pickle('./data/2020_06_27/general.p')
shoting    = pd.read_pickle('./data/2020_06_27/shoting.p')
passing    = pd.read_pickle('./data/2020_06_27/passing.p')
pass_types = pd.read_pickle('./data/2020_06_27/pass_types.p')
possession = pd.read_pickle('./data/2020_06_27/possession.p')
defense    = pd.read_pickle('./data/2020_06_27/defense.p')
gk         = pd.read_pickle('./data/2020_06_27/gk.p')
gk_adv     = pd.read_pickle('./data/2020_06_27/gk_adv.p')
playtime   = pd.read_pickle('./data/2020_06_27/playing_time.p')

## Remove Empty Rows

In [3]:
def remove_empty_rows(df):
    df = df.loc[df[('Unnamed: 1_level_0', 'Player')] != 'Player']
    return df

In [4]:
general    = remove_empty_rows(general) 
shoting    = remove_empty_rows(shoting)
passing    = remove_empty_rows(passing)
pass_types = remove_empty_rows(pass_types)
possession = remove_empty_rows(possession)
defense    = remove_empty_rows(defense)
gk         = remove_empty_rows(gk)
gk_adv     = remove_empty_rows(gk_adv)

## Remove multiplicated columns 

Some columns were repeated in generated dataframes, so let's ger rid of them

In [5]:
cols_todrop = [('Unnamed: 0_level_0', 'Rk'),
 ('Unnamed: 2_level_0', 'Nation'),
 ('Unnamed: 3_level_0', 'Pos'),
 ('Unnamed: 4_level_0', 'Squad'),
 ('Unnamed: 5_level_0', 'Comp'),
 ('Unnamed: 6_level_0', 'Age'),
 ('Unnamed: 7_level_0', 'Born'),
 ('Unnamed: 8_level_0', '90s')]

In [6]:
dataframes = [
                shoting   ,
                passing   ,
                pass_types,
                possession,
                defense   ,
                gk        ,
                gk_adv,
                playtime
]

In [7]:
for df in dataframes:
    df.drop(labels=cols_todrop, axis=1, errors='ignore', inplace=True)
    df.drop(columns=[x for x in df.columns if 'Matches' in x], inplace=True)

## Set player name as index 

In [8]:
colplayer = ( 'Unnamed: 1_level_0',  'Player')

In [9]:
for df in dataframes:
    df.set_index(colplayer, inplace=True)

In [10]:
general.set_index(colplayer, inplace=True)

### prepare to merge datasets 

I want to count number of columns in all dataframes before I merge them into one

46

In [11]:
df_columns_len = []
for df in dataframes:
    df_columns_len.append(len(df.columns))

In [12]:
sum(df_columns_len)

168

### Merge datasets

#### Field Players

In [13]:
t = pd.merge(general, shoting, left_index=True, right_index=True)

In [14]:
t1 = pd.merge(t, playtime, left_index=True, right_index=True)

In [15]:
m2 = pd.merge(t1, passing, left_index=True, right_index=True)

In [16]:
m3 = pd.merge(m2, pass_types, left_index=True, right_index=True)

In [17]:
m4 = pd.merge(m3, possession, left_index=True, right_index=True)

In [18]:
m5 = pd.merge(m4, defense, left_index=True, right_index=True)

In [19]:
result = m5

In [20]:
result.to_pickle('./data/2020_06_27/merged_data.p')

#### Goalkeepers

In [21]:
gkm = pd.merge(gk, gk_adv, left_index=True, right_index=True)

In [22]:
gkm.to_pickle('./data/2020_06_27/merged_gk.p')

## Convert to numeric values

In [23]:
df = result

In [57]:
[x for x in df.columns]

[('Unnamed: 0_level_0', 'Rk'),
 ('Unnamed: 2_level_0', 'Nation'),
 ('Unnamed: 3_level_0', 'Pos'),
 ('Unnamed: 4_level_0', 'Squad'),
 ('Unnamed: 5_level_0', 'Comp'),
 ('Unnamed: 6_level_0', 'Age'),
 ('Unnamed: 7_level_0', 'Born'),
 ('Playing Time_x', 'MP'),
 ('Playing Time_x', 'Starts'),
 ('Playing Time_x', 'Min'),
 ('Performance', 'Gls'),
 ('Performance', 'Ast'),
 ('Performance', 'PK'),
 ('Performance', 'PKatt'),
 ('Performance', 'CrdY'),
 ('Performance', 'CrdR'),
 ('Per 90 Minutes', 'Gls'),
 ('Per 90 Minutes', 'Ast'),
 ('Per 90 Minutes', 'G+A'),
 ('Per 90 Minutes', 'G-PK'),
 ('Per 90 Minutes', 'G+A-PK'),
 ('Expected_x', 'xG'),
 ('Expected_x', 'npxG'),
 ('Expected_x', 'xA'),
 ('Per 90 Minutes', 'xG'),
 ('Per 90 Minutes', 'xA'),
 ('Per 90 Minutes', 'xG+xA'),
 ('Per 90 Minutes', 'npxG'),
 ('Per 90 Minutes', 'npxG+xA'),
 ('Unnamed: 30_level_0', 'Matches'),
 ('Standard', 'Gls'),
 ('Standard', 'PK'),
 ('Standard', 'PKatt'),
 ('Standard', 'Sh'),
 ('Standard', 'SoT'),
 ('Standard', 'FK'),
 ('

In [24]:
df.drop(columns=[x for x in df.columns if 'Matches' in x], inplace=True)

In [25]:
non_num_cols = [(   'Unnamed: 2_level_0',  'Nation'),
            (   'Unnamed: 3_level_0',     'Pos'),
            (   'Unnamed: 4_level_0',   'Squad'),
            (   'Unnamed: 5_level_0',    'Comp')]

In [26]:
for column in df.columns:
    if column not in non_num_cols:
        df[column] = pd.to_numeric(df[column])

# columns cleaning

In [27]:
#position
positions = df[('Unnamed: 3_level_0', 'Pos')].str.split(',', expand=True)
df[('Position', 'Pos')] = positions[0]
df[('Position', 'Alt')] = positions[1]

In [28]:
df[('info', 'Player')] = df.index

In [29]:
#league - get rid of country name at the begining
df[('info', 'league')] = [x.split(' ', 1)[1] for x in df[('Unnamed: 5_level_0', 'Comp')]]

In [67]:
[x for x in df.columns]

[('Unnamed: 0_level_0', 'Rk'),
 ('Unnamed: 2_level_0', 'Nation'),
 ('Unnamed: 3_level_0', 'Pos'),
 ('Unnamed: 4_level_0', 'Squad'),
 ('Unnamed: 5_level_0', 'Comp'),
 ('Unnamed: 6_level_0', 'Age'),
 ('Unnamed: 7_level_0', 'Born'),
 ('Playing Time_x', 'MP'),
 ('Playing Time_x', 'Starts'),
 ('Playing Time_x', 'Min'),
 ('Performance', 'Gls'),
 ('Performance', 'Ast'),
 ('Performance', 'PK'),
 ('Performance', 'PKatt'),
 ('Performance', 'CrdY'),
 ('Performance', 'CrdR'),
 ('Per 90 Minutes', 'Gls'),
 ('Per 90 Minutes', 'Ast'),
 ('Per 90 Minutes', 'G+A'),
 ('Per 90 Minutes', 'G-PK'),
 ('Per 90 Minutes', 'G+A-PK'),
 ('Expected_x', 'xG'),
 ('Expected_x', 'npxG'),
 ('Expected_x', 'xA'),
 ('Per 90 Minutes', 'xG'),
 ('Per 90 Minutes', 'xA'),
 ('Per 90 Minutes', 'xG+xA'),
 ('Per 90 Minutes', 'npxG'),
 ('Per 90 Minutes', 'npxG+xA'),
 ('Standard', 'Gls'),
 ('Standard', 'PK'),
 ('Standard', 'PKatt'),
 ('Standard', 'Sh'),
 ('Standard', 'SoT'),
 ('Standard', 'FK'),
 ('Standard', 'SoT%'),
 ('Standard', 'Sh

In [30]:
df.to_pickle('./data/2020_06_27/merged_fieldplayers.p')